1. How many orders are cancelled and what’s the total value?

2. What’s the percentage of orders cancelled before payment and after payment?

3. Is cancellation rate higher for certain payment channels?

4. How many percent of orders are re-ordered from the same shop, and how long after cancellation do they re-order (mean)? 

Information about the dataset

All calculations should exclude orders from Shop ID 111111111.

Order Status:

- Paid = 2, 11, 12, 13
- Unpaid and Cancelled = 6
- Paid then Cancelled = 7, 8, 9, 10

In [157]:
import pandas as pd 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [158]:
file = "Training Workshop Data File - Sample Orders.xlsx"

df_order = pd.read_excel(open(file, 'rb'), sheet_name="Order Data")  
df_pay   = pd.read_excel(open(file, 'rb'), sheet_name="Payments Data")  



df_order.head()
df_order.shape
df_order.info()
df_pay.head()
df_pay.shape

,orderid,shopid,userid,checkoutid,total_price,currency,status,created_time,payment_time,cancel_time
0,6505222765,900457733,100223545,1044269436,147.168235,SGD,2,14/11/2019 21:33,2019-11-16 12:54:00,NaT
1,6135926959,586311226,100223545,1211270092,33.169735,SGD,2,13/12/2019 19:02,2019-12-13 21:33:00,NaT
2,5513066971,949679556,100223545,1676514901,24.112195,SGD,2,31/12/2019 17:51,2019-12-31 17:55:00,NaT
3,5123701563,894942828,100223545,1211270092,75.913637,SGD,6,13/12/2019 19:02,NaT,2019-12-15 21:38:00
4,7311653202,389950494,100223545,1880489412,14.455363,SGD,6,22/12/2019 14:47,NaT,2019-12-25 11:44:00


(7987, 10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7987 entries, 0 to 7986
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   orderid       7987 non-null   int64         
 1   shopid        7987 non-null   int64         
 2   userid        7987 non-null   int64         
 3   checkoutid    7987 non-null   int64         
 4   total_price   7987 non-null   float64       
 5   currency      7987 non-null   object        
 6   status        7987 non-null   int64         
 7   created_time  7987 non-null   object        
 8   payment_time  6284 non-null   datetime64[ns]
 9   cancel_time   2628 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(5), object(2)
memory usage: 624.1+ KB


,checkoutid,total_price,currency,payment_method,payment_time
0,123016653,1.176036,SGD,Credit Card,2019-12-12 19:03:00
1,128375081,3.461700,SGD,Credit Card,2019-12-28 00:13:00
2,150417433,3.188444,SGD,Credit Card,2019-12-10 11:30:00
3,169495571,1.664013,SGD,Credit Card,2019-11-05 19:11:00
4,205419935,1.244183,SGD,Credit Card,2019-11-12 02:05:00


(7838, 5)

In [159]:
#exclude orders from Shop ID 111111111. 
df_order = df_order[df_order["shopid"]!=111111111]


1. How many orders are cancelled and what’s the total value?

In [160]:
#cancelled orders

df_cancelled = df_order[df_order["status"].isin([6,7,8,9,10])]
print("Cancelled Orders : {:,}".format(df_cancelled["orderid"].nunique()))
print("Sum of Cancelled Orders in Amount : ${:,.0f}".format(df_cancelled["total_price"].sum()))

Cancelled Orders : 2,555
Sum of Cancelled Orders in Amount : $194,769


2. What’s the percentage of orders cancelled before payment and after payment?

In [161]:
df_cancel_status = df_cancelled.groupby("status")["orderid"].count().reset_index()
df_cancel_status



,status,orderid
0,6,1703
1,7,205
2,8,192
3,9,222
4,10,233


In [162]:
df_cancel_before = df_cancel_status[df_cancel_status["status"]==6]["orderid"].sum()
df_cancel_after  = df_cancel_status[df_cancel_status["status"]!=6]["orderid"].sum()

all_orders= df_cancel_status["orderid"].sum()

# print(df_cancel_before)
# print(df_cancel_after)

print("#Orders Cancelled Before {:,}, ({:.1%})".format(df_cancel_before, df_cancel_before/all_orders))
print("#Orders Cancelled After  {:,}, ({:.1%})".format(df_cancel_after, df_cancel_after/all_orders))



#Orders Cancelled Before 1,703, (66.7%)
#Orders Cancelled After  852, (33.3%)


3. Is cancellation rate higher for certain payment channels?

In [163]:
df_merge = df_order.merge(df_pay, on="checkoutid", how="left")
print("Rows, Columns: {},{}".format(df_merge.shape[0], df_merge.shape[1]))
df_merge.head()

Rows, Columns: 7911,14


,orderid,shopid,userid,checkoutid,total_price_x,currency_x,status,created_time,payment_time_x,cancel_time,total_price_y,currency_y,payment_method,payment_time_y
0,6505222765,900457733,100223545,1044269436,147.168235,SGD,2,14/11/2019 21:33,2019-11-16 12:54:00,NaT,NaN,NaN,NaN,NaT
1,6135926959,586311226,100223545,1211270092,33.169735,SGD,2,13/12/2019 19:02,2019-12-13 21:33:00,NaT,NaN,NaN,NaN,NaT
2,5513066971,949679556,100223545,1676514901,24.112195,SGD,2,31/12/2019 17:51,2019-12-31 17:55:00,NaT,24.112195,SGD,Credit Card,2019-12-31 17:55:00
3,5123701563,894942828,100223545,1211270092,75.913637,SGD,6,13/12/2019 19:02,NaT,2019-12-15 21:38:00,NaN,NaN,NaN,NaT
4,7311653202,389950494,100223545,1880489412,14.455363,SGD,6,22/12/2019 14:47,NaT,2019-12-25 11:44:00,14.455363,SGD,Credit Card,NaT


In [164]:
# - Paid = 2, 11, 12, 13
# - Unpaid and Cancelled = 6
# - Paid then Cancelled = 7, 8, 9, 10

status_paid =[2,11,12,13]

df_merge.groupby(["status","payment_method"])["orderid"].count().reset_index()




,status,payment_method,orderid
0,2,Bank Transfer,199
1,2,Credit Card,1072
2,2,ShopeePay,53
3,6,Bank Transfer,244
4,6,Credit Card,1338
5,6,ShopeePay,87
6,7,Bank Transfer,26
7,7,Credit Card,167
8,7,ShopeePay,8
9,8,Bank Transfer,32


In [165]:
def check_cancelled(df):
    status = df["status"]
    if status in status_paid :
        return "not cancelled"
    else:
        return "cancelled"


df_merge["is_cancelled"] = df_merge.apply(check_cancelled, axis="columns") 
df_merge.head()

,orderid,shopid,userid,checkoutid,total_price_x,currency_x,status,created_time,payment_time_x,cancel_time,total_price_y,currency_y,payment_method,payment_time_y,is_cancelled
0,6505222765,900457733,100223545,1044269436,147.168235,SGD,2,14/11/2019 21:33,2019-11-16 12:54:00,NaT,NaN,NaN,NaN,NaT,not cancelled
1,6135926959,586311226,100223545,1211270092,33.169735,SGD,2,13/12/2019 19:02,2019-12-13 21:33:00,NaT,NaN,NaN,NaN,NaT,not cancelled
2,5513066971,949679556,100223545,1676514901,24.112195,SGD,2,31/12/2019 17:51,2019-12-31 17:55:00,NaT,24.112195,SGD,Credit Card,2019-12-31 17:55:00,not cancelled
3,5123701563,894942828,100223545,1211270092,75.913637,SGD,6,13/12/2019 19:02,NaT,2019-12-15 21:38:00,NaN,NaN,NaN,NaT,cancelled
4,7311653202,389950494,100223545,1880489412,14.455363,SGD,6,22/12/2019 14:47,NaT,2019-12-25 11:44:00,14.455363,SGD,Credit Card,NaT,cancelled


In [166]:
df_payment_cancel = df_merge[df_merge["is_cancelled"]=="cancelled"].groupby(["is_cancelled","payment_method"])["orderid"].agg(["count"]).reset_index()
df_payment_cancel

,is_cancelled,payment_method,count
0,cancelled,Bank Transfer,372
1,cancelled,Credit Card,2014
2,cancelled,ShopeePay,121


In [167]:
n_cancelled = sum(df_payment_cancel["count"])
print(n_cancelled)


df_payment_cancel[df_payment_cancel["is_cancelled"]=="cancelled"]
df_payment_cancel["count"] = df_payment_cancel["count"].astype("float64")
df_cancel_pct = df_payment_cancel.groupby(level=0).sum()/n_cancelled

df_cancel_pct = round(df_cancel_pct*100,1)
df_cancel_pct.rename({"count":"%"} ,axis=1, inplace=True)
df_cancel_pct

print("most of the cancelled transactions are from credit cards")

2507


,is_cancelled,payment_method,count
0,cancelled,Bank Transfer,372
1,cancelled,Credit Card,2014
2,cancelled,ShopeePay,121


,%
0,14.8
1,80.3
2,4.8


most of the cancelled transactions are from credit cards


WIP

4. How many percent of orders are re-ordered from the same shop, and how long after cancellation do they re-order (mean)? 

In [168]:
df_merge.head()

,orderid,shopid,userid,checkoutid,total_price_x,currency_x,status,created_time,payment_time_x,cancel_time,total_price_y,currency_y,payment_method,payment_time_y,is_cancelled
0,6505222765,900457733,100223545,1044269436,147.168235,SGD,2,14/11/2019 21:33,2019-11-16 12:54:00,NaT,NaN,NaN,NaN,NaT,not cancelled
1,6135926959,586311226,100223545,1211270092,33.169735,SGD,2,13/12/2019 19:02,2019-12-13 21:33:00,NaT,NaN,NaN,NaN,NaT,not cancelled
2,5513066971,949679556,100223545,1676514901,24.112195,SGD,2,31/12/2019 17:51,2019-12-31 17:55:00,NaT,24.112195,SGD,Credit Card,2019-12-31 17:55:00,not cancelled
3,5123701563,894942828,100223545,1211270092,75.913637,SGD,6,13/12/2019 19:02,NaT,2019-12-15 21:38:00,NaN,NaN,NaN,NaT,cancelled
4,7311653202,389950494,100223545,1880489412,14.455363,SGD,6,22/12/2019 14:47,NaT,2019-12-25 11:44:00,14.455363,SGD,Credit Card,NaT,cancelled


In [180]:
df_user_orders = df_merge.groupby(["shopid","userid"])[["orderid"]].agg(["count"]).reset_index()

df_user_cancelled = df_merge[df_merge["is_cancelled"]=="cancelled"].groupby(["shopid"])[["orderid","userid"]].agg(["count"]).reset_index()
df_user_cancelled.rename({"count":"cancelled"} ,axis=1, inplace=True)

df_user_orders.head()
df_user_cancelled.head()

,shopid,userid,orderid
,,,count
0,100264691,916554285,1
1,100464073,611554042,1
2,100531365,283568769,1
3,100728385,667880870,1
4,100823882,176830897,1


,shopid,orderid,userid
,,cancelled,cancelled
0,100464073,1,1
1,100823882,1,1
2,101650594,1,1
3,101835385,1,1
4,102095669,1,1


In [172]:
#merge the 2

df_user_2 = df_user_orders.merge(df_user_cancelled, on=["shopid"],how="left")
df_user_2.head()
#(df_user_2["count"]>df_user2["cancelled"])&

/Users/g/anaconda3/envs/py38/lib/python3.8/site-packages/pandas/core/generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


shopid orderid          
               count cancelled
0  100264691       1       NaN
1  100464073       1       1.0
2  100531365       1       NaN
3  100728385       1       NaN
4  100823882       1       1.0

In [179]:
df_merge

,orderid,shopid,userid,checkoutid,total_price_x,currency_x,status,created_time,payment_time_x,cancel_time,total_price_y,currency_y,payment_method,payment_time_y,is_cancelled
1958,7347201304,999927017,327046956,1555538470,105.411726,SGD,13,23/11/2019 01:24,2019-11-23 13:11:00,NaT,105.411726,SGD,ShopeePay,2019-11-23 13:11:00,not cancelled
4074,4521197700,999820287,560189570,1129779569,29.680577,SGD,10,16/10/2019 20:03,2019-10-18 01:15:00,2019-10-18 21:05:00,29.680577,SGD,Credit Card,2019-10-18 01:15:00,cancelled
1230,1382065621,999736550,241476014,1795202716,42.941444,SGD,13,29/11/2019 21:09,2019-12-01 19:57:00,NaT,42.941444,SGD,Credit Card,2019-12-01 19:57:00,not cancelled
6172,7613607926,999612455,801490536,1389090019,82.704692,SGD,12,24/11/2019 22:37,2019-11-25 03:20:00,NaT,82.704692,SGD,Credit Card,2019-11-25 03:20:00,not cancelled
3726,7923134997,999467686,523758062,1694279965,14.241792,SGD,13,05/10/2019 00:16,2019-10-06 07:45:00,NaT,14.241792,SGD,Bank Transfer,2019-10-06 07:45:00,not cancelled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,9510247544,100823882,176830897,1637848990,32.569997,SGD,6,08/10/2019 16:28,NaT,2019-10-11 16:16:00,32.569997,SGD,Credit Card,NaT,cancelled
4959,3563024034,100728385,667880870,1493749164,5.262632,SGD,11,10/10/2019 16:25,2019-10-10 16:28:00,NaT,5.262632,SGD,Credit Card,2019-10-10 16:28:00,not cancelled
1596,1526942168,100531365,283568769,1215585382,38.871672,SGD,11,01/10/2019 11:44,2019-10-03 11:20:00,NaT,38.871672,SGD,Credit Card,2019-10-03 11:20:00,not cancelled
4499,4506038116,100464073,611554042,1059688064,16.779622,SGD,6,05/12/2019 13:37,NaT,2019-12-07 05:53:00,16.779622,SGD,Credit Card,NaT,cancelled
